In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
! pip install tensorflow-gpu==2.3.0 # for multi-GPU training

In [7]:
# !python3 model.py
!python3 main.py
# !python3 dataset.py
# !python3 config.py

2021-06-21 08:14:48.610573: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
tf.__version__:  2.3.0
2021-06-21 08:14:50.253887: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-06-21 08:14:50.382292: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-06-21 08:14:50.382347: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-10-15-172): /proc/driver/nvidia/version does not exist
2021-06-21 08:14:50.383412: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropria

In [24]:
from config import *
import pandas as pd
import sys
import numpy as np

# MAX_TEST_SEQ_LEN = 228

for chunk_idx, chunk in enumerate(pd.read_csv(TEST_DATA_PATH, chunksize=1, nrows=NUM_TEST_SAMPLES_QUANTITATIVE)): 
    
    print(type(chunk), chunk.shape)
    chunk_array = np.squeeze(chunk.to_numpy())
    print(chunk_array.shape)
    
    user_id = chunk_array[0]
    test_seq_len = chunk_array[1]
    print("test_seq_len: ", test_seq_len)        
    song_id_list = chunk_array[2:2+MAX_TEST_SEQ_LEN][-test_seq_len:]
    print("song_id_list: ", song_id_list)
    print(song_id_list[-10:])

    cut = len(song_id_list)//2
    
    xs = []
    
    
    if chunk_idx == 0:
        sys.exit()

<class 'pandas.core.frame.DataFrame'> (1, 458)
(458,)
test_seq_len:  5
song_id_list:  [3241  134   93  692  110]
[3241  134   93  692  110]


SystemExit: 